In [1]:
"""
Person: playing individual, can be the computer dealer or the human player
"""
class Person():
    """Initialize object"""
    def __init__(self, name):
        # name of the person
        self.name = name
        
        # sum of cards the person has for a round (default: 0 because hand is empty)
        self.value = 0
        
        # cards the person has in their hand for a round (default: empty list)
        self.hand = []
    
    """Add cards to hand"""
    def add_cards(self, cards):
        # add to hand based on the number of cards
        if type(cards) == type([]):
            # merge with existing hand if more than 1 card
            self.hand.extend(cards)
        else:
            # add 1 card to hand
            self.hand.append(cards)
    
    """Compute value of cards in hand"""
    def compute_value(self):
        value = 0
        
        # get the sum total of all card values in hand
        for card in self.hand:
            if type(card.value) == type([]):
                card.choose_value()
            
            # add card value to total
            value += card.value
        
        # update hand value
        self.value = value
    
    """Show cards in hand"""
    def show_hand(self, limit=0):
        print("{}'s cards".format(self.name))
        
        # limit number of cards to display
        if limit == 0:
            # for player, show all cards and their total value
            # list all cards in hand
            for i, card in enumerate(self.hand):
                print("  [{}] {}".format(i+1, card))
        else:
            # for dealer, show only the first card
            # limit the card to show based on the limit
            for i, card in enumerate(self.hand):
                if limit > 0:
                    print("  [{}] {}".format(i+1, card))
                    limit -= 1
                else:
                    break
    
    """Ask whether to hit or stay"""
    def ask_action(self):
        # define choices
        choices = ['hit', 'stay']
        
        # ask until chosen a valid option
        while True:
            # ask choice
            choice = int(input("{}, do you want to [1] hit or [2] stay? ".format(self.name)))
            
            # verify choice
            if choice not in [1, 2]:
                # choice not within the valid options
                print("Invalid choice, try again")
            else:
                # choice valid
                break
        
        # return choice
        return choices[choice - 1]
    
    """Return person's name and hand value"""
    def __str__(self):
        return "Name: {} | card/s: {} value: {}".format(self.name, len(self), self.value)
    
    """Return person's no. of cards in hand"""
    def __len__(self):
        return len(self.hand)


In [2]:
"""
Dealer (Person): the computer dealer
"""
class Dealer(Person):
    """Initialize object"""
    def __init__(self, name='Dealer'):
        # initialize Dealer as Person
        Person.__init__(self, name)


In [3]:
"""
Player (Person): the human player
"""
class Player(Person):
    """Initialize object"""
    def __init__(self, name, bankroll):
        # name of the player (default: 'Player')
        self.name = name
        
        # amount of credits the player has (default: 0.0)
        self.bankroll = bankroll
        
        # amount of credits the player is waging for the round (default: 0.0)
        self.bet = 0.0
        
        # initialize Player as Person
        Person.__init__(self, name)
    
    """Ask player the amount to bet for the round"""
    def place_bet(self):
        # check bankroll if not empty
        if self.bankroll > 0:
            # bankroll has enough amount
            
            # continue asking player until right amount is bet
            while True:
                # ask player their bet
                bet = float(input("{}, place your bet (bankroll: {}): ".format(self.name, self.bankroll)))
                
                # check bankroll limits
                if bet > self.bankroll:
                    # bet must be within bankroll amount
                    print("Bet must not exceed bankroll")
                elif bet < 1:
                    # bet must not be less than 1
                    print("Bet must not be less than 1")
                else:
                    print("Bet placed: {} | Bankroll: {}".format(bet, self.bankroll))
                    
                    self.bet = bet
                    break
        else:
            # do not ask player since bankroll is not enough
            print("Bankroll not enough")
    
    """Update bankroll after winning bet"""
    def wins(self):
        print("{} wins! Hand value is {}.".format(self.name, self.value))
        
        self.bankroll += self.bet
        
    """Update bankroll after losing bet"""
    def loses(self):
        print("{} loses. Hand value is {}.".format(self.name, self.value))
        
        self.bankroll -= self.bet
    
    """Return details about the player"""
    def __str__(self):
        return Person.__str__(self) + " | bankroll: {} bet: {}".format(self.bankroll, self.bet)


In [4]:
"""
Card: playing card in a normal deck
"""
class Card():
    """Class attributes"""
    suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
    ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
    values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10, 'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': [1, 11]}
    
    """Initialize object"""
    def __init__(self, suit, rank, value):
        # hearts, diamonds, spades, clubs
        self.suit = suit
        
        # 2, 3, ..., 9, 10, Jack, Queen, King, Ace
        self.rank = rank
        
        # numeric value of a card; can be an integer or array (for Ace)
        # 2 to 10 count as their face value
        # J, Q, K count as 10
        # Ace counts as 1 or 11 depending on the player's choice
        self.value = value
    
    """Ask player to choose the value of the Ace card for their hand (1 or 11)"""
    def choose_value(self):
        # prepare prompt and value to ask player
        prompt = "Choose Ace card value ({}): ".format(" or ".join([str(value) for value in self.value]))
        value = 0
        
        # continue asking player until correct value is entered
        while True:
            # check entered card value
            if value not in self.value:
                # ask again if entered value is incorrect
                value = int(input(prompt))
            else:
                # correct value
                break
        
        # return chosen value
        self.value = value
    
    """Return details about the card"""
    def __str__(self):
        return "{} of {}".format(self.rank, self.suit)


In [5]:
from random import shuffle

"""
Deck: normal collection of 52 cards
"""
class Deck():
    # tells whether the deck is shuffled (default: False)
    is_shuffled = False
    
    """Initialize class"""
    def __init__(self):
        # list of all 52 cards in a normal deck
        self.cards = []
        
        # fill up deck with the basic 52 cards
        for suit in Card.suits:
            for rank in Card.ranks:
                card = Card(suit, rank, Card.values[rank])
                self.cards.append(card)
    
    """Draw the top card from the deck"""
    def deal_one(self):
        return self.cards.pop()
    
    """Shuffle the cards in the deck"""
    def shuffle(self):
        shuffle(self.cards)
        self.is_shuffled = True
    
    """Return details about the deck"""
    def __str__(self):
        return "{} card/s in deck (shuffled: {})".format(len(self.cards), str(self.is_shuffled))
    
    """Return the total number of cards left in the deck"""
    def __len__(self):
        return len(self.cards)


In [6]:
# prepare and shuffle deck
deck = Deck()
deck.shuffle()

# prepare player
name = input("Enter player's name: ")
bankroll = float(input("Enter player's bankroll: "))
player = Player(name, bankroll)

# prepare dealer
name = input("Enter dealer's name: ")
dealer = Dealer(name)

# play until deck runs out of cards or player has no more money in the bankroll
while len(deck) > 0 and player.bankroll > 0:
    # empty hands of player and dealer
    player.hand = []
    dealer.hand = []
    
    # draw two cards each for both player and dealer
    player.add_cards([deck.deal_one(), deck.deal_one()])
    dealer.add_cards([deck.deal_one(), deck.deal_one()])
    
    # ask player to place their bet
    player.place_bet()
    
    # show 1 card of dealer's hand
    dealer.show_hand(1)
    
    # ask player to hit or stay while hand value does not exceed 21
    while player.value <= 21:
        # show player's hand and compute their value
        player.show_hand()
        player.compute_value()
        print("  Value: {}".format(player.value))
        
        # ask player to hit or stay
        action = player.ask_action()
        
        # hit to deal one card, else stay
        if action == 'hit':
            # add card from the deck to player's hand and show them
            player.add_cards(deck.deal_one())
            player.compute_value()
        else:
            # stay, break from loop
            break
    
    # player decides to stay and hand value does not exceed 21
    if action == 'stay' and player.value <= 21:
        # show dealer hand and compute their value
        dealer.show_hand()
        dealer.compute_value()
        print("  Value: {}".format(dealer.value))
        
        # continue hitting until hand value is more than 17
        while dealer.value <= 17:
            # hit to draw one card from deck and add to hand value
            dealer.add_cards(deck.deal_one())
            dealer.compute_value()
        
        # show dealer's hand
        dealer.show_hand()
        print("  Value: {}".format(dealer.value))
    
    # decide outcome for the round
    if player.value > 21:
        # player loses (dealer wins) because player's hand value exceeds 21
        player.loses()
    elif dealer.value > 21:
        # player wins (dealer loses) because dealer's hand value exceeds 21
        player.wins()
    elif player.value > dealer.value:
        # player wins (dealer loses) because player's hand value is more than the dealer's
        player.wins()
    elif player.value < dealer.value:
        # player loses (dealer wins) because dealer's hand value is more than the player's
        player.loses()
    else:
        # draw because both dealer and player has the same hand value
        print("Draw! {}'s value {} is similar to {}'s value {}".format(player.name, player.value, dealer.name, dealer.value))
    
    # ask player to continue playing or exit game
    choice = 'X'
    
    # continue asking until Y or N is entered
    while choice.upper() not in ['Y', 'N']:
        choice = input("Continue playing (Y or N)? ")
    
    # player chooses to exit game
    if choice.upper() == 'N':
        print("Thanks for playing!")
        print("Player {} earnings: {}".format(player.name, player.bankroll))
        break
    else:
        print("Game continues! {}'s bankroll is {}".format(player.name, player.bankroll))
else:
    # end game
    if len(deck) == 0:
        # when deck is empty
        print("Game has ended! Deck is already empty!")
    elif player.bankroll == 0:
        # when player's bankroll is empty
        print("Game has ended! Player has no more money!")


Enter player's name: Argem
Enter player's bankroll: 100
Enter dealer's name: Gerald
Argem, place your bet (bankroll: 100.0): 20
Bet placed: 20.0 | Bankroll: 100.0
Gerald's cards
  [1] King of Clubs
Argem's cards
  [1] Nine of Diamonds
  [2] Four of Diamonds
  Value: 13
Argem, do you want to [1] hit or [2] stay? 1
Argem's cards
  [1] Nine of Diamonds
  [2] Four of Diamonds
  [3] Five of Clubs
  Value: 18
Argem, do you want to [1] hit or [2] stay? 2
Gerald's cards
  [1] King of Clubs
  [2] Three of Clubs
  Value: 13
Gerald's cards
  [1] King of Clubs
  [2] Three of Clubs
  [3] Nine of Hearts
  Value: 22
Argem wins! Hand value is 18.
Continue playing (Y or N)? N
Thanks for playing!
Player Argem earnings: 120.0
